In [1]:
try:
  # We must install required packages if we are in Google Colab
  import google.colab
  %pip install roboticstoolbox-python>=1.0.2
  %pip install qpsolvers[quadprog]
  COLAB = True
except:
  # We are not in Google Colab
  COLAB = False
  # Apply custon style to notebook
  from IPython.core.display import HTML
  import pathlib
  styles_path = pathlib.Path(pathlib.Path().absolute(), "style", "style.css")
  styles = open(styles_path, "r").read()
  HTML(f"<style>{styles}</style>")

# 6.0 Advanced Numerical Inverse Kinematics

$\large{\text{Manipulator Differential Kinematics}} \\ \large{\text{Part II: Acceleration and Advanced Applications}}$

$\text{By Jesse Haviland and Peter Corke}$

<br>

The sections of the Tutorial paper related to this notebook are listed next to each contents entry.
It is beneficial to read these sections of the paper before attempting the notebook Section.

### Contents

[6.1 Numerical Inverse Kinematics Utility Class](#utility)
* Resolved-Rate Motion Control
  * Angle-axis error component
* Numerical Inverse Kinematics

[6.2 Newton-Raphson (NR) Method](#nr)
* Numerical Inverse Kinematics

[6.2 Newton-Raphson (NR) Method](#nr)
* Numerical Inverse Kinematics

[6.3 Gauss-Newton (GN) Method](#gn)
* Numerical Inverse Kinematics

[6.4 Levenberg-Marquardt (LM) Method](#lm)
* Numerical Inverse Kinematics

[6.5 Visualise IK Solutions](#vis)

[6.6 Numerical IK Comparison](#exp)

[6.7 Fast IK Solvers with the Robotics Toolbox](#rtb)

[6.8 Robotics Toolbox IK Speed Comparison](#speed)


In [2]:
# We will do the imports required for this notebook here

# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# ansitable is a great package for printing tables in a terminal
from ansitable import ANSITable

# python mechanisms to create abstract classes
from abc import ABC, abstractmethod

# a package for creating dynamic progress bars
from progress.bar import Bar

# swift is a lightweight browser-based simulator which comes with the toolbox
from swift import Swift

# spatialgeometry is a utility package for dealing with geometric objects
import spatialgeometry as sg

# provides sleep functionaly
import time

# Typing utilities
from typing import Optional

# A set of quadratic programming sovlers
import qpsolvers as qp

# Allows us make method wrappers
from functools import wraps

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

<br>

<a id='utility'></a>

### 4.1 Numerical Inverse Kinematics Utility Class
---

Inverse kinematics deals is the problem of determining the corresponding joint coordinates, given some end-effector pose. Numerical inverse kinematics as opposed to analytical inverse kinematics uses an iterative technique.

We will start this Notebook by making an abstract base class called `IK`. This class implements everything needed to support a numerical inverse kinematics solver. It stores some general parameters typically used by any IK solver and also has some data structures useful for large-scale experiments. The idea behind the `IK` class is that a specific method class will inherit `IK` and implement the `step` method.

An important point to note is that numerical IK methods are subject to local minima and in such some will fail to converge on the solution. The choice of the initial joint configuration $\bf{q}_0$ is important.

An alternative approach is to re-start an IK problem with a new random $\bf{q}_0$ after a few $20 \sim 50$ iterations rather than persist with a single search attempt with $500 \sim 5000$ iterations. This is a simple but effective method of performing a global search for the IK solution. We can adjust this using the iteration limit `ilimit` and search limit `slimit` kwargs on the `IK` class.

In the Tutorial, we use a quadratic error term to describe the error between the current end-effector pose and desired end-effector pose. This error is consistent across each method we discuss.

Each method seeks to minimise the error function

\begin{align*}
    E = \frac{1}{2} \bf{e}^{\top} \bf{W}_e \bf{e}
\end{align*}

where $\bf{e} \in \mathbb{R}^6$ is the position and angle-axis error vector (see Part 1 Notebook 3), and $\bf{W}_e = \text{diag}(\bf{w_e})(\bf{w_e} \in \mathbb{R}^n_{\geq0})$ is a diagonal weighting matrix which prioritises the corresponding error term. The `IK` class below takes a `we` vector in the `init` method and calculates the error vector `e` and quadratic error term `E` through the `error` method.

In [3]:
class IK(ABC):
    """
    An abstract super class which provides basic functionality to perform numerical inverse
    kinematics (IK). Superclasses can inherit this class and implement the solve method.

    This class also provides a mechanism to collect data on performance for large scale
    experiments.
    """

    def __init__(
        self,
        name: str = "IK Solver",
        ilimit: int = 30,
        slimit: int = 100,
        tol: float = 1e-6,
        we: np.ndarray = np.ones(6),
        problems: int = 1000,
        reject_jl: bool = True,
        λΣ: float=0.0,
        λm: float=0.0, 
        ps: float=0.1,
        pi: Optional[np.ndarray]=None,
    ):
        """
        name: The name of the IK algorithm
        ilimit: How many iterations are allowed within a search before a new search is started
        slimit: How many searches are allowed before being deemed unsuccessful
        tol: Maximum allowed residual error E
        we: A 6 vector which assigns weights to Cartesian degrees-of-freedom
        problems: Total number of IK problems within the experiment
        reject_jl: Reject solutions with joint limit violations
        λΣ: The gain for joint limit avoidance. Setting to 0.0 will remove this completely from the solution
        λm: The gain for maximisation. Setting to 0.0 will remove this completely from the solution
        ps: The minimum angle/distance (in radians or metres) in which the joint is allowed to approach to its limit
        pi: The influence angle/distance (in radians or metres) in null space motion becomes active
        """

        # Solver parameters
        self.name = name
        self.slimit = slimit
        self.ilimit = ilimit
        self.tol = tol
        self.We = np.diag(we)
        self.reject_jl = reject_jl
        self.λΣ = λΣ
        self.λm = λm
        self.ps = ps
        self.pi = pi

        # Solver results
        self.success = np.zeros(problems)
        self.searches = np.zeros(problems)
        self.iterations = np.zeros(problems)
        self.valid_jl = np.zeros(problems)
        self.times = np.zeros(problems)

        # initialise with NaN
        self.searches[:] = np.nan
        self.iterations[:] = np.nan
        self.success[:] = np.nan
        self.valid_jl[:] = np.nan
        self.times[:] = np.nan

    def solve(self, ets: rtb.ETS, Tep: np.ndarray, q0: np.ndarray):
        """
        This method will attempt to solve the IK problem and obtain joint coordinates
        which result the the end-effector pose Tep.

        The method returns a tuple:
        q: The joint coordinates of the solution (ndarray). Note that these will not
            be valid if failed to find a solution
        success: True if a solution was found (boolean)
        iterations: The number of iterations it took to find the solution (int)
        searches: The number of searches it took to find the solution (int)
        residual: The residual error of the solution (float)
        jl_valid: True if joint coordinates q are within the joint limits
        total_t: The total time spent within the step method
        """

        # Iteration count
        i = 0
        total_i = 0
        total_t = 0.0

        for search in range(self.slimit):
            q = q0[search].copy()
            
            while i <= self.ilimit:
                i += 1

                # Attempt a step
                try:
                    t, E, q = self.step(ets, Tep, q)

                    # Acclumulate total time
                    total_t += t
                except np.linalg.LinAlgError:
                    # Abandon search and try again
                    break

                # Check if we have arrived
                if E < self.tol:

                    # Wrap q to be within +- 180 deg
                    # If your robot has larger than 180 deg range on a joint
                    # this line should be modified in incorporate the extra range
                    q = (q + np.pi) % (2 * np.pi) - np.pi

                    # Check if we have violated joint limits
                    jl_valid = self.check_jl(ets, q)

                    if not jl_valid and self.reject_jl:
                        # Abandon search and try again
                        break
                    else:
                        return q, True, total_i + i, search + 1, E, jl_valid, total_t

            total_i += i
            i = 0

        # If we make it here, then we have failed
        return q, False, np.nan, np.nan, E, np.nan, np.nan

    def error(self, Te: np.ndarray, Tep: np.ndarray):
        """
        Calculates the engle axis error between current end-effector pose Te and
        the desired end-effector pose Tep. Also calulates the quadratic error E
        which is weighted by the diagonal matrix We.

        Returns a tuple:
        e: angle-axis error (ndarray in R^6)
        E: The quadratic error weighted by We
        """
        e = rtb.angle_axis(Te, Tep)
        E = 0.5 * e @ self.We @ e

        return e, E

    def check_jl(self, ets: rtb.ETS, q: np.ndarray):
        """
        Checks if the joints are within their respective limits

        Returns a True if joints within feasible limits otherwise False
        """

        # Loop through the joints in the ETS
        for i in range(ets.n):

            # Get the corresponding joint limits
            ql0 = ets.qlim[0, i]
            ql1 = ets.qlim[1, i]

            # Check if q exceeds the limits
            if q[i] < ql0 or q[i] > ql1:
                return False

        # If we make it here, all the joints are fine
        return True

    @abstractmethod
    def step(self, ets: rtb.ETS, Tep: np.ndarray, q: np.ndarray):
        """
        Superclasses will implement this method to perform a step of the implemented
        IK algorithm
        """
        pass


We will also make a method which will wrap each our `step` methods to record how long each call to `step` takes.

In [4]:
def timing(func):
    @wraps(func)
    def wrap(*args, **kw):
        t_start = time.time()
        E, q = func(*args, **kw)
        t_end = time.time()
        t = t_end - t_start
        return t, E, q
    return wrap

<br>

<a id='2'></a>

### 4.2 Null-Space Motion in IK
---

We can greatly improve the solvability of our solvers when they are trying to avoid the joint limits by using the null-space projection devised by Baur [6]

\begin{align*} 
    \bf{q}_{k+1} &=
    \bf{q}_k +
    {\bf{J}(\bf{q}_k)}^{+} \bf{e}_k +
    \bf{q}_{null} \\
    \bf{q}_{null} &= 
    \Big(
        \left(
            \bf{1}_n - \bf{J}(\bf{q})^{+} \bf{J}(\bf{q})
        \right)
        \Big(
            \frac{1}{\lambda_\Sigma}\bf{\Sigma}
        \Big)
    \Big) \\
    \bf{\Sigma} &=
    \left\{
        \begin{matrix}
            \sum_{i=1}^n
            \dfrac{(q_i - \bar{q}_{M_i})^2}
                  {(q_{M_i} - \bar{q}_{M_i})^2}
            & q_i \geq \bar{q}_{M_i} \\
            \sum_{i=1}^n
            \dfrac{(q_i - \bar{q}_{m_i})^2}
                  {(q_{m_i} - \bar{q}_{m_i})^2}
            & q_i \leq \bar{q}_{m_i} \\
            0 & \text{else}
        \end{matrix}
    \right.
\end{align*}

where the maximum and minimum joint angles for a joint are specified with $q_{M_i}$ and $q_{m_i}$, while the maximum and minimum joint angle threshold is specified by $\bar{q}_{M_i}$ and $\bar{q}_{m_i}$. Once this threshold is passed, further progress toward the joint limit is penalised by $\bf{\Sigma}$. While this addition can help avoid joint limits, it does not guarantee joint limit avoidance. The term $\lambda_\Sigma \in \mathbb{R}^+$ is a gain which adjusts how aggressively the joint limit is avoided by $\bf{\Sigma}$.

Lets make a method to calculate $\bf{\Sigma}$ in Python

In [5]:
def null_Σ(ets: rtb.ETS, q: np.ndarray, ps: float, pi: Optional[np.ndarray]):
    """
    Formulates a relationship between joint limits and the joint velocity.
    When this is projected into the null-space of the differential kinematics
    to attempt to avoid exceeding joint limits

    q: The joint coordinates of the robot
    ps: The minimum angle/distance (in radians or metres) in which the joint is
        allowed to approach to its limit
    pi: The influence angle/distance (in radians or metres) in which the velocity
        damper becomes active

    returns: Σ 
    """

    # If pi wasn't supplied, set it to some default value
    if pi is None:
        pi = 0.3 * np.ones(ets.n)

    # Add cost to going in the direction of joint limits, if they are within
    # the influence distance
    Σ = np.zeros((ets.n, 1))

    for i in range(ets.n):
        qi = q[i]
        ql0 = ets.qlim[0, i]
        ql1 = ets.qlim[1, i]

        if qi - ql0 <= pi[i]:
            Σ[i, 0] = (
                -np.power(((qi - ql0) - pi[i]), 2) / np.power((ps - pi[i]), 2)
            )
        if ql1 - qi <= pi[i]:
            Σ[i, 0] = (
                np.power(((ql1 - qi) - pi[i]), 2) / np.power((ps - pi[i]), 2)
            )

    return -Σ

Furthermore, we can add the manipulability Jacobian to the null-space term as we did in Part 2 Notebook 4 to get

\begin{align*}
\bf{q}_{null} &= 
\Big(
    \left(
        \bf{1}_n - \bf{J}(\bf{q})^{+} \bf{J}(\bf{q})
    \right)
    \Big(
        \frac{1}{\lambda_\Sigma}\bf{\Sigma} + \frac{1}{\lambda_m} \bf{J}_m(\bf{q})
    \Big)
\Big)
\end{align*}

where $\lambda_m \in \mathbb{R}^+$ is a gain which adjusts how aggressively the manipulability is to be maximised.

We will incorporate this as an optional addition to each of our IK solvers we implemented in Part 1 Notebook 4.

If we set the `IK` class kwarg `λΣ` to a non-zero value, then $\bf{\Sigma}$ will be projected into the null-space, set to 0.0 if you do not wish for it to be a part of the solver. Similarly, set the `IK` class kwarg `λm` to a non-zero value and $\bf{J}_m$ will be projected into the null-space, set to 0.0 if you do not wish for it to be a part of the solver. Both `λΣ` and `λm` can be non-zero at the same time if desired.


In [6]:
def calc_qnull(
        ets: rtb.ETS,
        q: np.ndarray,
        J: np.ndarray,
        λΣ: float,
        λm: float,
        ps: float,
        pi: Optional[np.ndarray]
    ):
    """
    Calculates the desired null-space motion according to the gains λΣ and λm.
    This is a helper method that is used within the `step` method of an IK solver

    Returns qnull: the desired null-space motion
    """

    qnull_grad = np.zeros(ets.n)
    qnull = np.zeros(ets.n)

    # Add the joint limit avoidance if the gain is above 0
    if λΣ > 0:
        Σ = null_Σ(ets, q, ps, pi)
        qnull_grad += (1.0 / λΣ * Σ).flatten()

    # Add the manipulability maximisation if the gain is above 0
    if λm > 0:
        Jm = ets.jacobm(q)
        qnull_grad += (1.0 / λm * Jm).flatten()

    # Calculate the null-space motion
    if λΣ > 0 or λΣ > 0:
        null_space = (np.eye(ets.n) - np.linalg.pinv(J) @ J)
        qnull = null_space @ qnull_grad

    return qnull.flatten()

<br>

<a id='nr'></a>

### 4.2 Newton-Raphson (NR) Method
---

The Newton-Raphson (NR) method seeks to minimise the error $E$ by iterating upon the following

\begin{align*}
    \bf{q}_{k+1} = \bf{q}_k + {^0\bf{J}(\bf{q}_k)}^{-1} \bf{e}_k.
\end{align*}

When using the NR method, the initial joint coordinates $q_0$, should correspond to a non-singular manipulator pose, since it uses the manipulator Jacobian.
When the the problem is solvable, it converges very quickly.
However, this method frequently fails to converge on the goal.

Most linear algebra libraries (including the Python `numpy` library) implement the pseudoinverse using singular value decomposition, which is robust to singular matrices. Therefore, we can this solver more robust by using the pseudoinverse instead of the normal inverse. 

The above equation requires the Jacobian to be square and non-singular. For a redundant manipulator (>6 DoF) or for more robust results to singularities we can use the pseudoinverse. We can use the pseudoinverse by supplying `pinv=True` when initialising the `NR` class.

In [7]:
class NR(IK):
    def __init__(self, pinv=False, **kwargs):
        super().__init__(**kwargs)
        self.pinv = pinv

        self.name = f"NR (pinv={pinv})"

        if self.λΣ > 0.0:
            self.name += ' Σ'

        if self.λm > 0.0:
            self.name += ' Jm'

    @timing
    def step(self, ets: rtb.ETS, Tep: np.ndarray, q: np.ndarray):
        Te = ets.eval(q)
        e, E = self.error(Te, Tep)

        J = ets.jacob0(q)

        # Null-space motion
        qnull = calc_qnull(ets, q, J, self.λΣ, self.λm, self.ps, self.pi)

        if self.pinv:
            q += np.linalg.pinv(J) @ e + qnull
        else:
            q += np.linalg.inv(J) @ e + qnull

        return E, q

<br>

<a id='gn'></a>

### 4.3 Gauss-Newton (GN) Method
---

We can improve the solvability of the NR method by using the Gauss-Newton (GN) method

\begin{align*}
    \bf{q}_{k+1} &= \bf{q}_k +
    \left(
    {\bf{J}(\bf{q}_k)}^\top
    \bf{W}_e \
    {\bf{J}(\bf{q}_k)}
    \right)^{-1}
    \bf{g}_k \\
    \bf{g}_k &=
    {\bf{J}(\bf{q}_k)}^\top
    \bf{W}_e
    \bf{e}_k
\end{align*}

where $\bf{J} = {^0\bf{J}}$ is the base-frame manipulator Jacobian. If $\bf{J}(\bf{q}_k)$ is non-singular, and $\bf{W}_e = \bf{1}_n$, then the above provides the pseudoinverse solution. However, if $\bf{J}(\bf{q}_k)$ is singular, the above can not be computed and the GN solution is infeasible.

Most linear algebra libraries (including the Python `numpy` library) implement the pseudoinverse using singular value decomposition, which is robust to singular matrices. Therefore, we can this solver more robust by using the pseudoinverse instead of the normal inverse. We can use the pseudoinverse by supplying `pinv=True` when initialising the `GN` class.

In [8]:
class GN(IK):
    def __init__(self, pinv=False, **kwargs):
        super().__init__(**kwargs)
        self.pinv = pinv

        self.name = f"GN (pinv={pinv})"

        if self.λΣ > 0.0:
            self.name += ' Σ'

        if self.λm > 0.0:
            self.name += ' Jm'

    @timing
    def step(self, ets: rtb.ETS, Tep: np.ndarray, q: np.ndarray):
        Te = ets.eval(q)
        e, E = self.error(Te, Tep)

        J = ets.jacob0(q)
        g = J.T @ self.We @ e

        # Null-space motion
        qnull = calc_qnull(ets, q, J, self.λΣ, self.λm, self.ps, self.pi)

        if self.pinv:
            q += np.linalg.pinv(J.T @ self.We @ J) @ g + qnull
        else:
            q += np.linalg.inv(J.T @ self.We @ J) @ g + qnull

        return E, q

<br>

<a id='lm'></a>

### 4.4 Levenberg-Marquardt (LM) Method
---

We can further improve the solvability though the Levenberg-Marquardt (LM) Method

\begin{align*}
    \bf{q}_{k+1} 
    &= 
    \bf{q}_k +
    \left(
        \bf{A}_k
    \right)^{-1}
    \bf{g}_k \\
    %
    \bf{A}_k
    &=
    {\bf{J}(\bf{q}_k)}^\top
    \bf{W}_e \
    {\bf{J}(\bf{q}_k)}
    +
    \bf{W}_n
\end{align*}

where $\bf{W}_n = \text{diag}(\bf{w_n})(\bf{w_n} \in \mathbb{R}^n_{>0})$ is a diagonal damping matrix. The damping matrix ensures that $\bf{A}_k$ is non-singular and positive definite. The performance of the LM method largely depends on the choice of $\bf{W}_n$.

#### Wampler's Method

Wampler proposed $\bf{w_n}$ to be a constant. We provide the variable `λ` as a kwarg to the `LM_Wampler` class to adjust the constant.

In [9]:
class LM_Wampler(IK):
    def __init__(self, λ=1.0, **kwargs):
        super().__init__(**kwargs)
        
        self.name = f"LM (Wampler λ={λ})"
        self.λ = λ

        if self.λΣ > 0.0:
            self.name += ' Σ'

        if self.λm > 0.0:
            self.name += ' Jm'

    @timing
    def step(self, ets: rtb.ETS, Tep: np.ndarray, q: np.ndarray):
        Te = ets.eval(q)
        e, E = self.error(Te, Tep)

        Wn = self.λ * np.eye(ets.n)
        J = ets.jacob0(q)
        g = J.T @ self.We @ e

        # Null-space motion
        qnull = calc_qnull(ets, q, J, self.λΣ, self.λm, self.ps, self.pi)

        q += np.linalg.inv(J.T @ self.We @ J + Wn) @ g + qnull

        return E, q

#### Chan's Method

Chan proposed

\begin{align*}
    \bf{W}_n
    &=
    \lambda E_k \bf{1}_n
\end{align*}

where $\lambda$ is a constant which reportedly does not have much influence on performance. We provide the variable `λ` as a kwarg to the `LM_Chan` class to adjust the weighting term.

In [10]:
class LM_Chan(IK):
    def __init__(self, λ=1.0, **kwargs):
        super().__init__(**kwargs)
        
        self.name = f"LM (Chan λ={λ})"
        self.λ = λ

        if self.λΣ > 0.0:
            self.name += ' Σ'

        if self.λm > 0.0:
            self.name += ' Jm'

    @timing
    def step(self, ets: rtb.ETS, Tep: np.ndarray, q: np.ndarray):
        Te = ets.eval(q)
        e, E = self.error(Te, Tep)

        Wn = self.λ * E * np.eye(ets.n)
        J = ets.jacob0(q)
        g = J.T @ self.We @ e

        # Null-space motion
        qnull = calc_qnull(ets, q, J, self.λΣ, self.λm, self.ps, self.pi)

        q += np.linalg.inv(J.T @ self.We @ J + Wn) @ g + qnull

        return E, q


#### Sugihara's Method

Sugihara proposed

\begin{align*}
    \bf{W}_n
    &=
    E_k \bf{1}_n + \text{diag}(\hat{\bf{w}}_n)
\end{align*}

where $\hat{\bf{w}}_n \in \mathbb{R}^n$, $\hat{w}_{n_i} = l^2 \sim 0.01 l^2$, and $l$ is the length of a typical link within the manipulator.
We provide the variable `λ` as a kwarg to the `LM_Sugihara` class to adjust the value of $w_n$.

In [11]:
class LM_Sugihara(IK):
    def __init__(self, name="LM (Sugihara)", λ=1.0, **kwargs):
        super().__init__(name, **kwargs)

        self.name = f"LM (Sugihara λ={λ})"
        self.λ = λ

        if self.λΣ > 0.0:
            self.name += ' Σ'

        if self.λm > 0.0:
            self.name += ' Jm'

    @timing
    def step(self, ets: rtb.ETS, Tep: np.ndarray, q: np.ndarray):
        Te = ets.eval(q)
        e, E = self.error(Te, Tep)

        Wn = E * np.eye(ets.n) + self.λ * np.eye(ets.n)
        J = ets.jacob0(q)
        g = J.T @ self.We @ e

        # Null-space motion
        qnull = calc_qnull(ets, q, J, self.λΣ, self.λm, self.ps, self.pi)

        q += np.linalg.inv(J.T @ self.We @ J + Wn) @ g + qnull

        return E, q

<br>

<a id='qp'></a>

### 4.5 Quadratic Programming Method
---

The addition of null-space motion provides much better results for inverse kinematics when trying to avoid joint limits, but it is only available on redundant robots. For improved constrained inverse kinematics, we can use the augmented QP with slack from Part 2 Notebook 5 with

\begin{align*}
    \bf{q}_{k+1} = \bf{q}_{k} + \dot{\bf{q}}.
\end{align*}

where the QP is defined as

\begin{align*} 
    \min_x \quad f_o(\bf{x}) &= \frac{1}{2} \bf{x}^\top \mathcal{Q} \bf{x}+ \mathcal{C}^\top \bf{x}, \\ 
    \text{subject to} \quad \mathcal{J} \bf{x} &= \bf{\nu},  \\
    \mathcal{A} \bf{x} &\leq \mathcal{B},  \\
    \bf{x}^- &\leq \bf{x} \leq \bf{x}^+ 
\end{align*}

with

\begin{align*}
    \bf{x} &= 
    \begin{pmatrix}
        \dot{\bf{q}} \\ \bf{\delta}
    \end{pmatrix} \in \mathbb{R}^{(n+6)}  \\
    \mathcal{Q} &=
    \begin{pmatrix}
        \lambda_q \bf{1}_{n} & \mathbf{0}_{6 \times 6} \\ \mathbf{0}_{n \times n} & \lambda_\delta \bf{1}_{6}
    \end{pmatrix} \in \mathbb{R}^{(n+6) \times (n+6)} \\
    \mathcal{J} &=
    \begin{pmatrix}
        \bf{J}(\bf{q}) & \bf{1}_{6}
    \end{pmatrix} \in \mathbb{R}^{6 \times (n+6)} \\
    \mathcal{C} &= 
    \begin{pmatrix}
        \bf{J}_m \\ \bf{0}_{6 \times 1}
    \end{pmatrix} \in \mathbb{R}^{(n + 6)} \\
    \mathcal{A} &= 
    \begin{pmatrix}
        \bf{1}_{n \times n + 6} \\
    \end{pmatrix} \in \mathbb{R}^{(l + n) \times (n + 6)} \\
    \mathcal{B} &= 
    \eta
    \begin{pmatrix}
        \frac{\rho_0 - \rho_s}
                {\rho_i - \rho_s} \\
        \vdots \\
        \frac{\rho_n - \rho_s}
                {\rho_i - \rho_s} 
    \end{pmatrix} \in \mathbb{R}^{n} \\
    \bf{x}^{-, +} &= 
    \begin{pmatrix}
        \dot{\bf{q}}^{-, +} \\
        \bf{\delta}^{-, +}
    \end{pmatrix} \in \mathbb{R}^{(n+6)},
\end{align*}

where
$\bf{\delta} \in \mathbb{R}^6$ is the slack vector,
$\lambda_\delta \in \mathbb{R}^+$ is a gain term which adjusts the cost of the norm of the slack vector in the optimiser,
$\dot{\bf{q}}^{-,+}$ are the minimum and maximum joint velocities, and 
$\dot{\bf{\delta}}^{-,+}$ are the minimum and maximum slack velocities.

In [12]:
class QP(IK):
    def __init__(self, name="QP", λj=1.0, λs=1.0, **kwargs):
        super().__init__(name, **kwargs)

        self.name = f"QP (λj={λj}, λs={λs})"
        self.λj = λj
        self.λs = λs

        if self.λΣ > 0.0:
            self.name += ' Σ'

        if self.λm > 0.0:
            self.name += ' Jm'

    @timing
    def step(self, ets: rtb.ETS, Tep: np.ndarray, q: np.ndarray):
        Te = ets.eval(q)
        e, E = self.error(Te, Tep)
        J = ets.jacob0(q)

        # Quadratic component of objective function
        Q = np.eye(ets.n + 6)

        # Joint velocity component of Q
        Q[: ets.n, : ets.n] *= self.λj

        # Slack component of Q
        Q[ets.n :, ets.n :] = self.λs * (1 / np.sum(np.abs(e))) * np.eye(6)

        # The equality contraints
        Aeq = np.concatenate((J, np.eye(6)), axis=1)
        beq = e.reshape((6,))

        # The inequality constraints for joint limit avoidance
        if self.λΣ > 0.0:
            Ain = np.zeros((ets.n + 6, ets.n + 6))
            bin = np.zeros(ets.n + 6)

            # Form the joint limit velocity damper
            Ain_l = np.zeros((ets.n, ets.n))
            Bin_l = np.zeros(ets.n)

            for i in range(ets.n):
                ql0 = ets.qlim[0, i]
                ql1 = ets.qlim[1, i]

                if ql1 - q[i] <= self.pi[i]:
                    Bin_l[i] = ((ql1 - q[i]) - self.ps) / (self.pi[i] - self.ps)
                    Ain_l[i, i] = 1

                if q[i] - ql0 <= self.pi[i]:
                    Bin_l[i] = -(((ql0 - q[i]) + self.ps) / (self.pi[i] - self.ps))
                    Ain_l[i, i] = -1

            Ain[: ets.n, : ets.n] = Ain_l
            bin[: ets.n] =  (1.0 / self.λΣ) * Bin_l
        else:
            Ain = None
            bin = None
        
        # Manipulability maximisation
        if self.λm > 0.0:
            Jm = ets.jacobm(q).reshape((ets.n,))
            c = np.concatenate(((1.0 / self.λm) * -Jm, np.zeros(6)))
        else:
            c = np.zeros(ets.n + 6)

        xd = qp.solve_qp(Q, c, Ain, bin, Aeq, beq, lb=None, ub=None, solver='quadprog')

        q += xd[: ets.n]

        return E, q

<br>

<a id='vis'></a>

### 4.5 Visualise IK Solutions
---

_Note: This **section** of the notebook must be run locally. Due to how Swift operates, this section will not run on Google Colab_

We are going to try out some of our methods and visualise the results in Swift

In [13]:
# Make our robot
robot = rtb.models.Valkyrie()

# Extract the waist to right index finger links as an ETS
start = None
end = "rightIndexFingerPitch3Link"
ets = robot.ets(start, end)

# Keep the original joint indices for later
og_ets_jindices = [et.jindex for et in ets.joints()]

# Reset the joint indices to count from 0 to ets.n
for i, et in enumerate(ets.joints()):
    et.jindex = i

# Calculate parameters for joint limit avoidance
q_range = ets.qlim[1, :] - ets.qlim[0, :]
pi = 0.5 * q_range
ps = 0.001

# Make an axes to visualise our current end-effector pose
ee_axes = sg.Axes(0.1)

# Make an axes to visualise our desired end-effector pose
goal_axes = sg.Axes(0.1)

def make_ik_env():
    # Make the environment
    env = Swift()

    # Launch the simulator
    env.launch(realtime=True, browser="notebook")

    # Add our robot to the simulator envionment
    env.add(robot)

    # Add them to the environment
    env.add(ee_axes)
    env.add(goal_axes)

    return env


We will establish some goal poses and a general visualise method.

In [14]:
# Number of problems
problems = 10

# Iteration and seach limits
ilimit = 30
slimit = 100

# random valid q values for q0
q0 = ets.random_q(slimit)

# random valid q values which will define Tep
q_Tep = ets.random_q(problems)

# Our desired end-effector poses
Tep = np.zeros((problems, 4, 4))

for i in range(problems):
    Tep[i] = ets.eval(q_Tep[i])

def visualise_ik(solver, env):
    """
    A method which will test the solver over each pose in Tep
    and visualise the results in Swift
    """
    for pose in Tep:

        # Solver for Tep
        q, success, iterations, searches, residual, q_valid, _ = solver.solve(ets, pose, q0)

        # print the results
        print(f"Successful: {success}, iterations: {iterations}, searches: {searches}, residual: {residual}, q_valid: {q_valid}")

        ### visualise the results
        # set the robots joint coordinates to the solution q
        # make sure q has no NaNs first
        if not np.any(np.isnan(q)):
            robot.q[og_ets_jindices] = q

            # set the end-effector axes to the end-effector pose at the solution q
            # ee_axes.T = robot.fkine(robot.q, end=end).A
            ee_axes.T = ets.fkine(q)

        # set the goal axes to Tep pose
        goal_axes.T = pose

        # step the environment to view the changes
        env.step(0)

        # sleep for 2 seconds to view the results
        time.sleep(2)

       

Lets test out the NR method first. Since we are using the Panda robot which has 7 DoF and is redundant we must set `pinv=True`.

In [15]:
if not COLAB:
    # Make our solver
    solver = NR(pinv=True, ilimit=30, slimit=100, problems=problems, reject_jl=True, λΣ=2.0, λm=10.0, ps=ps, pi=pi)

    # Visualise the solver
    env = make_ik_env()
    visualise_ik(solver, env)

Successful: True, iterations: 46, searches: 4, residual: 5.789347025754664e-07, q_valid: True
Successful: True, iterations: 48, searches: 4, residual: 5.216091433856326e-07, q_valid: True
Successful: True, iterations: 26, searches: 2, residual: 3.7753175620126285e-07, q_valid: True
Successful: True, iterations: 14, searches: 1, residual: 7.82472567891041e-07, q_valid: True
Successful: True, iterations: 11, searches: 1, residual: 3.5631907638920104e-07, q_valid: True
Successful: True, iterations: 131, searches: 10, residual: 9.834891580968926e-07, q_valid: True
Successful: True, iterations: 32, searches: 3, residual: 5.127516516736984e-08, q_valid: True
Successful: True, iterations: 48, searches: 4, residual: 6.140630585887269e-07, q_valid: True
Successful: True, iterations: 51, searches: 4, residual: 2.819849983727459e-07, q_valid: True
Successful: True, iterations: 66, searches: 6, residual: 4.5285358496157236e-07, q_valid: True


Visualise the LM Chan method

In [16]:
if not COLAB:
    # Make our solver
    solver = LM_Chan(λ=0.1, ilimit=30, slimit=100, problems=problems, reject_jl=True, λΣ=2.0, λm=10.0, ps=ps, pi=pi)

    # Visualise the solver
    env = make_ik_env()
    visualise_ik(solver, env)

Successful: True, iterations: 56, searches: 4, residual: 6.102753054033355e-07, q_valid: True
Successful: True, iterations: 8, searches: 1, residual: 3.05853210234488e-07, q_valid: True
Successful: True, iterations: 12, searches: 1, residual: 9.113892039556061e-07, q_valid: True
Successful: True, iterations: 16, searches: 1, residual: 3.6643612104369494e-07, q_valid: True
Successful: True, iterations: 7, searches: 1, residual: 4.425576649499674e-07, q_valid: True
Successful: True, iterations: 24, searches: 2, residual: 8.067953484184202e-07, q_valid: True
Successful: True, iterations: 51, searches: 3, residual: 3.0835952695748077e-07, q_valid: True
Successful: True, iterations: 11, searches: 1, residual: 8.605401694620144e-07, q_valid: True
Successful: True, iterations: 11, searches: 1, residual: 7.077552621998414e-07, q_valid: True
Successful: True, iterations: 27, searches: 2, residual: 6.344576472196917e-07, q_valid: True


Feel free to modify the above cells to try out any of the other methods. You may have noticed that many of the solutions violated the joint limits of the panda. In Part II of the Tutorial we will address this issue.

<br>

<a id='exp'></a>

### 4.6 Numerical IK Comparison
---

In this Section we will perform a large scale experiment to compare the performance of each IK method.

This Notebook is intended to be set up as an IK playground. Feel free to modify, extend, or create your own methods as shown above. Then, simply add your solver to the `solvers` list below and text how it compares to the existing methods. Also feel free to try out different robots!

As shown above, the `ETS` class has its own method to generate random valid joint coordinate vectors. However, to make the table in the paper reporoducible, we replicate the method here.

In [17]:
def random_q(ets: rtb.ETS, i: int = 1) -> np.ndarray:
    """
    Generate a random valid joint configuration

    :param i: number of configurations to generate

    Generates a random q vector within the joint limits defined by
    `ets.qlim`.
    """

    if i == 1:
        q = np.zeros(ets.n)

        for i in range(ets.n):
            q[i] = np.random.uniform(ets.qlim[0, i], ets.qlim[1, i])

    else:
        q = np.zeros((i, ets.n))

        for j in range(i):
            for i in range(ets.n):
                q[j, i] = np.random.uniform(ets.qlim[0, i], ets.qlim[1, i])

    return q

In the following cell, we make a general IK experiment method.

In [18]:
def ik_experiment(solvers, Tep, ets):
    """
    This method takes a list of solvers and a list of desired end-effector poses
    and will run the complete IK experiment while collecting data and will print
    a table once finished
    """
        
    with Bar(message="Prog", max=problems, check_tty=False, hide_cursor=False) as bar:
        for i, pose in enumerate(Tep):

            for solver in solvers:
                (
                    _,
                    solver.success[i],
                    solver.iterations[i],
                    solver.searches[i],
                    _,
                    solver.valid_jl[i],
                    solver.times[i],
                ) = solver.solve(ets, pose, q0)

            bar.next()
        bar.finish()

    # Calculate the time per iteration of each of the solvers
    t_iter = [np.nansum(solver.times) / np.nansum(solver.iterations) for solver in solvers]

    # Normalise the time to a fraction of the quickest time per iteration within the solvers
    t_iter_norm = t_iter / np.min(t_iter)

    # Make a table to visualise the results
    print(f"\nNumerical Inverse Kinematics Methods Compared over {problems} problems\n")

    table = ANSITable(
        "Method",
        "Mean Iter.",
        "Median Iter.",
        "Infeasible",
        "Infeasible %",
        "Mean Searches",
        "Max Searches",
        "jl Violations",
        "Time per Iter.",
        "Median Time",
        border="thin",
    )

    for solver, time in zip(solvers, t_iter_norm):
        table.row(
            solver.name,
            np.round(np.nanmean(solver.iterations), 2),
            np.nanmedian(solver.iterations),
            np.sum(np.isnan(solver.iterations)),
            np.round(np.sum(np.isnan(solver.iterations)) / problems * 100.0, 2),
            np.round(np.nanmean(solver.searches), 2),
            np.nanmax(solver.searches),
            np.sum(solver.valid_jl == False),
            np.round(time, 2),
            np.round(time * np.nanmedian(solver.iterations), 2),
        )

    table.print()


Then we define some default parameters which each of our experiments will use. The experiments will take a while to run. If you want results faster, reduce the number `problems`.

In [19]:
### Experiment parameters
# Number of problems to solve
problems = 10000

# Maximum slimit, we will preallocate the random q0 values so each
# solver uses the same set of random q0 values
max_slimit = 100

# Calculate parameters for joint limit avoidance
ps = 0.001

# Joint limit avoidance and manipulability maximisation gains
λΣ=2.0
λΣ_panda=100.0
λm=10.0
λm_qp=100.0
λm_qp_valk=1000.0

#### UR5 Experiment

Lets test our IK solvers on a 6 DoF UR5 robot.

What is shown below will recreate the data used for Table 1 in the Paper.

In [20]:
# Our robot and ETS
robot = rtb.models.UR5()
ets = robot.ets()

# Setting the seed will make it possible to regenrate Table 1
# in the paper (provided no ther values were changed).
np.random.seed(10)

# random valid q values which will define Tep
q_Tep = ets.random_q(problems)

# Our desired end-effector poses
Tep = np.zeros((problems, 4, 4))

for i in range(problems):
    Tep[i] = ets.eval(q_Tep[i])

# random valid q values for q0
q0 = ets.random_q(max_slimit)

# Calculate parameters for joint limit avoidance
q_range = ets.qlim[1, :] - ets.qlim[0, :]
pi = 0.5 * q_range

solvers_ur5 = [
    ### Not avoiding joint limits
    NR(pinv=True, problems=problems, reject_jl=False),
    LM_Chan(λ=0.1, problems=problems, reject_jl=False),
    ### No null
    LM_Wampler(λ=1e-4, problems=problems, λΣ=0.0, λm=0.0),
    LM_Chan(λ=0.1, problems=problems, λΣ=0.0, λm=0.0),
    LM_Sugihara(λ=0.0001, problems=problems, λΣ=0.0, λm=0.0),
    ### QP
    QP(λj=0.01, λs=1.0, problems=problems, λΣ=0.0, λm=λm_qp, ps=ps, pi=pi)
]

ik_experiment(solvers_ur5, Tep, ets)


Prog |################################| 10000/10000


Numerical Inverse Kinematics Methods Compared over 10000 problems

┌────────────────────────┬────────────┬──────────────┬────────────┬──────────────┬───────────────┬──────────────┬───────────────┬────────────────┬─────────────┐
│                 Method │ Mean Iter. │ Median Iter. │ Infeasible │ Infeasible % │ Mean Searches │ Max Searches │ jl Violations │ Time per Iter. │ Median Time │
├────────────────────────┼────────────┼──────────────┼────────────┼──────────────┼───────────────┼──────────────┼───────────────┼────────────────┼─────────────┤
│         NR (pinv=True) │      28.63 │         16.0 │          0 │          0.0 │          1.46 │         29.0 │             0 │           2.37 │       37.95 │
│        LM (Chan λ=0.1) │      15.52 │          8.0 │          0 │          0.0 │          1.21 │         14.0 │             0 │           1.41 │       11.27 │
│  LM (Wampler λ=0.0001) │      23.75 │         13.0 │          0 │          0.0 │          1.35 │         20.0 │             0

Because the UR5 has $\pm 360°$ rotation in each of its joints, every angle for every joint is reachable. Therefore, our advanced null-space and QP methods which push the joints away from their limits provide no benefit. This is reflected in the table above.

#### Panda Experiment

Lets test our IK solvers on a 7 DoF Panda robot.

What is shown below will recreate the data used for Table 2 in the Paper.

In [21]:
# Our robot and ETS
robot = rtb.models.Panda()
ets = robot.ets()

# Setting the seed will make it possible to regenrate Table 2
# in the paper (provided no ther values were changed).
np.random.seed(10)

# random valid q values which will define Tep
q_Tep = ets.random_q(problems)

# Our desired end-effector poses
Tep = np.zeros((problems, 4, 4))

for i in range(problems):
    Tep[i] = ets.eval(q_Tep[i])

# random valid q values for q0
q0 = ets.random_q(max_slimit)

# Calculate parameters for joint limit avoidance
q_range = ets.qlim[1, :] - ets.qlim[0, :]
pi = 0.5 * q_range

solvers_panda = [
    ### Not avoiding joint limits
    NR(pinv=True, problems=problems, reject_jl=False),
    LM_Chan(λ=0.1, problems=problems, reject_jl=False),
    ### No null
    NR(pinv=True, problems=problems, λΣ=0.0, λm=0.0),
    LM_Wampler(λ=1e-4, problems=problems, λΣ=0.0, λm=0.0),
    LM_Chan(λ=0.1, problems=problems, λΣ=0.0, λm=0.0),
    LM_Sugihara(λ=0.0001, problems=problems, λΣ=0.0, λm=0.0),
    ### λΣ
    NR(pinv=True, problems=problems, λΣ=λΣ, λm=0.0, ps=ps, pi=pi),
    LM_Wampler(λ=1e-4, problems=problems, λΣ=λΣ, λm=0.0, ps=ps, pi=pi),
    LM_Chan(λ=0.1, problems=problems, λΣ=λΣ_panda, λm=0.0, ps=ps, pi=pi),
    LM_Sugihara(λ=0.0001, problems=problems, λΣ=λΣ, λm=0.0, ps=ps, pi=pi),
    ### λΣ and λm
    NR(pinv=True, problems=problems, λΣ=λΣ_panda, λm=λm, ps=ps, pi=pi),
    LM_Wampler(λ=1e-4, problems=problems, λΣ=λΣ_panda, λm=λm, ps=ps, pi=pi),
    LM_Chan(λ=0.1, problems=problems, λΣ=λΣ_panda, λm=λm, ps=ps, pi=pi),
    LM_Sugihara(λ=0.0001, problems=problems, λΣ=λΣ_panda, λm=λm, ps=ps, pi=pi),
    ### QP
    QP(λj=0.01, λs=1.0, problems=problems, λΣ=λΣ, λm=λm_qp, ps=ps, pi=pi)
]

ik_experiment(solvers_panda, Tep, ets)

Prog |################################| 10000/10000


Numerical Inverse Kinematics Methods Compared over 10000 problems

┌────────────────────────────┬────────────┬──────────────┬────────────┬──────────────┬───────────────┬──────────────┬───────────────┬────────────────┬─────────────┐
│                     Method │ Mean Iter. │ Median Iter. │ Infeasible │ Infeasible % │ Mean Searches │ Max Searches │ jl Violations │ Time per Iter. │ Median Time │
├────────────────────────────┼────────────┼──────────────┼────────────┼──────────────┼───────────────┼──────────────┼───────────────┼────────────────┼─────────────┤
│             NR (pinv=True) │       27.6 │         16.0 │          0 │          0.0 │          1.41 │         13.0 │          6697 │           2.99 │       47.87 │
│            LM (Chan λ=0.1) │      11.91 │          8.0 │          0 │          0.0 │          1.12 │          7.0 │          5394 │           1.28 │       10.23 │
│             NR (pinv=True) │     137.67 │         80.0 │        103 │         1.03 │          7.39 │     

The Panda has more narrow joint limit range than the UR5. Therefore, there are joint configurations which are invalid as shown by the first two methods in the Table above. In this context, our null-space and QP methods start to provide benefits.

#### Valkyrie Experiment

Lets test our IK solvers on a 13 DoF section of the Valkyrie robot.

What is shown below will recreate the data used for Table 3 in the Paper.

In [22]:
# Our robot and ETS
robot = rtb.models.Valkyrie()

# Extract the waist to right index finger links as an ETS
start = None
end = "rightIndexFingerPitch3Link"
ets = robot.ets(start, end)

# Reset the joint indices to count from 0 to ets.n
for i, et in enumerate(ets.joints()):
    et.jindex = i

# Setting the seed will make it possible to regenrate Table 3
# in the paper (provided no ther values were changed).
np.random.seed(10)

# random valid q values which will define Tep
q_Tep = ets.random_q(problems)

# Our desired end-effector poses
Tep = np.zeros((problems, 4, 4))

for i in range(problems):
    Tep[i] = ets.eval(q_Tep[i])

# random valid q values for q0
q0 = ets.random_q(max_slimit)

# Calculate parameters for joint limit avoidance
q_range = ets.qlim[1, :] - ets.qlim[0, :]
pi = 0.5 * q_range



solvers_panda = [
    ### Not avoiding joint limits
    LM_Chan(λ=0.1, problems=problems, reject_jl=False),
    ### No null
    NR(pinv=True, problems=problems, λΣ=0.0, λm=0.0),
    LM_Chan(λ=0.1, problems=problems, λΣ=0.0, λm=0.0),
    ### λΣ
    NR(pinv=True, problems=problems, λΣ=λΣ, λm=0.0, ps=ps, pi=pi),
    LM_Wampler(λ=1e-4, problems=problems, λΣ=λΣ, λm=0.0, ps=ps, pi=pi),
    LM_Chan(λ=0.1, problems=problems, λΣ=λΣ, λm=0.0, ps=ps, pi=pi),
    LM_Sugihara(λ=0.0001, problems=problems, λΣ=λΣ, λm=0.0, ps=ps, pi=pi),
    ### λΣ and λm
    LM_Chan(λ=0.1, problems=problems, λΣ=λΣ, λm=λm, ps=ps, pi=pi),
    LM_Sugihara(λ=0.0001, problems=problems, λΣ=λΣ, λm=λm, ps=ps, pi=pi),
    ### QP
    QP(λj=0.01, λs=1.0, problems=problems, λΣ=λΣ, λm=λm_qp_valk, ps=ps, pi=pi)
]

ik_experiment(solvers_panda, Tep, ets)

Prog |################################| 10000/10000




Numerical Inverse Kinematics Methods Compared over 10000 problems

┌────────────────────────────┬────────────┬──────────────┬────────────┬──────────────┬───────────────┬──────────────┬───────────────┬────────────────┬─────────────┐
│                     Method │ Mean Iter. │ Median Iter. │ Infeasible │ Infeasible % │ Mean Searches │ Max Searches │ jl Violations │ Time per Iter. │ Median Time │
├────────────────────────────┼────────────┼──────────────┼────────────┼──────────────┼───────────────┼──────────────┼───────────────┼────────────────┼─────────────┤
│            LM (Chan λ=0.1) │       6.31 │          6.0 │          0 │          0.0 │           1.0 │          1.0 │          9542 │           3.75 │       22.51 │
│             NR (pinv=True) │     285.88 │        235.0 │       2791 │        27.91 │         34.57 │        100.0 │             0 │           1.08 │      253.16 │
│            LM (Chan λ=0.1) │     156.13 │         98.0 │       1765 │        17.65 │         25.22 │     

This section of the Valkyrie has joint limits which are even more narrow than the Panda. The solutions of IK methods (the first row of the Table above) which aren't trying to find solutions with feasible joint coordinates find a large 

here are joint configurations which are invalid as shown by the first two methods in the Table above. In this context, our null-space and QP methods start to provide benefits.

<br>

<a id='rtb'></a>

### 4.7 Fast IK Solvers with the Robotics Toolbox
---

In this section we will briefly demonstrate the IK solvers implemented in the Robotics Toolbox. 

The Toolbox has implemented some of the above IK techniques using C extensions and wrapped by python methods. The results generated from these methods will not perfectly replicate the above results as

* We can not use the same random number generator or seed
* The C implementations use different matrix inverse and pseudo-inverse calculators

So far, we have not implemented the null-space projections or the QP solver in C extensions but these will be included in future versions of the Robotics Toolbox.

Note that with these solvers, we can add the `reject_jl=True` kwarg for the solver to reject solutions which violate joint limits in a similar manner to how it has been implemented above. Using this kwarg, we will compare the implemented IK solvers within the Toolbox at finding IK solutions with achievable joint coordinates.

We make a simplified wrapper class for this experiment as we only wish to collect the data.

In [23]:
class RTB_IK:
    def __init__(self, name, solve, problems=problems):

        # Solver attributes
        self.name = name
        self.solve = solve

        # Solver results
        self.success = np.zeros(problems)
        self.searches = np.zeros(problems)
        self.iterations = np.zeros(problems)

        # initialise with NaN
        self.success[:] = np.nan
        self.searches[:] = np.nan
        self.iterations[:] = np.nan


We create the experiment parameters here

In [24]:
# Our robot and ETS
robot = rtb.models.Panda()
ets = robot.ets()

# Number of problems to solve
rtb_problems = 10000

# Cartesion DoF priority matrix
we = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

# random valid q values which will define Tep
q_rand = ets.random_q(rtb_problems)

# Our desired end-effector poses
Tep = np.zeros((rtb_problems, 4, 4))

for i in range(rtb_problems):
    Tep[i] = ets.eval(q_rand[i])

# Maximum iterations allowed in a search
ilimit = 30

# Maximum searches allowed per problem
slimit = 100

# Solution tolerance
tol = 1e-6

We create a list of solvers similar to Section 4.6 but use `lambda` funtions to point to the solver we wish to use. 

In [25]:
rtb_solvers = [
    RTB_IK(
        "Newton Raphson (pinv=True)",
        lambda Tep: ets.ik_NR(
            Tep,
            ilimit=ilimit,
            slimit=slimit,
            tol=tol,
            mask=we,
            pinv=True,
            joint_limits=True
        ),
        problems=rtb_problems,
    ),
    RTB_IK(
        "Gauss Newton (pinv=True)",
        lambda Tep: ets.ik_GN(
            Tep,
            ilimit=ilimit,
            slimit=slimit,
            tol=tol,
            mask=we,
            pinv=True,
            joint_limits=True
        ),
        problems=rtb_problems,
    ),
    RTB_IK(
        "LM Wampler 1e-4",
        lambda Tep: ets.ik_LM(
            Tep,
            ilimit=ilimit,
            slimit=slimit,
            tol=tol,
            mask=we,
            k=1e-4,
            joint_limits=True,
            method="wampler"
        ),
        problems=rtb_problems,
    ),
    RTB_IK(
        "LM Chan 0.1",
        lambda Tep: ets.ik_LM(
            Tep,
            q0=None,
            ilimit=ilimit,
            slimit=slimit,
            tol=tol,
            mask=we,
            k=0.1,
            joint_limits=True,
            method="chan"
        ),
        problems=rtb_problems,
    ),
    RTB_IK(
        "LM Sugihara 0.0001",
        lambda Tep: ets.ik_LM(
            Tep,
            ilimit=ilimit,
            slimit=slimit,
            tol=tol,
            mask=we,
            k=0.0001,
            joint_limits=True,
            method="sugihara"
        ),
        problems=rtb_problems,
    ),
]


Now we can run the experiment. This will be much faster than the Python implementations shown above.

In [26]:
with Bar(message="Prog", max=rtb_problems, check_tty=False, hide_cursor=False) as bar:
    for i, pose in enumerate(Tep):

        for solver in rtb_solvers:
            (
                _,
                solver.success[i],
                solver.iterations[i],
                solver.searches[i],
                _,
            ) = solver.solve(pose)

        bar.next()
    bar.finish()


# Make a table to visualise the results
print(f"\nNumerical Inverse Kinematics Methods Compared over {rtb_problems} problems\n")

table = ANSITable(
    "Method",
    "sLimit",
    "iLimit",
    "Mean Iterations",
    "Median Iterations",
    "Infeasible",
    "Infeasible %",
    "Mean Searches",
    "Max Searches",
    border="thin",
)

for solver in rtb_solvers:
    # print(solver.iterations)
    table.row(
        solver.name,
        slimit,
        ilimit,
        np.round(np.nanmean(solver.iterations), 2),
        np.nanmedian(solver.iterations),
        np.sum(np.isnan(solver.iterations)),
        np.round(np.sum(np.isnan(solver.iterations)) / rtb_problems * 100.0, 2),
        np.round(np.nanmean(solver.searches), 2),
        np.nanmax(solver.searches),
    )

table.print()

Prog |################################| 10000/10000


Numerical Inverse Kinematics Methods Compared over 10000 problems

┌───────────────────────────┬────────┬────────┬─────────────────┬───────────────────┬────────────┬──────────────┬───────────────┬──────────────┐
│                    Method │ sLimit │ iLimit │ Mean Iterations │ Median Iterations │ Infeasible │ Infeasible % │ Mean Searches │ Max Searches │
├───────────────────────────┼────────┼────────┼─────────────────┼───────────────────┼────────────┼──────────────┼───────────────┼──────────────┤
│Newton Raphson (pinv=True) │    100 │     30 │           572.3 │             356.5 │          0 │          0.0 │         30.25 │        101.0 │
│  Gauss Newton (pinv=True) │    100 │     30 │          563.94 │             362.0 │          0 │          0.0 │         29.92 │        101.0 │
│           LM Wampler 1e-4 │    100 │     30 │          501.13 │             312.0 │          0 │          0.0 │         27.56 │        101.0 │
│               LM Chan 0.1 │    100 │     30 │           57.3

<br>

<a id='speed'></a>

### 4.8 Robotics Toolbox IK Speed Comparison
---

In this last section we will compare the speed of some of the Robotics Toolbox IK implementations while rejecting solutions which have unreachable joint coordinates.

In [27]:
# Number of problems to solve
speed_problems = 10000

# Cartesion DoF priority matrix
we = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

# random valid q values which will define Tep
q_rand = ets.random_q(speed_problems)

# Our desired end-effector poses
Tep = np.zeros((speed_problems, 4, 4))

for i in range(speed_problems):
    Tep[i] = ets.eval(q_rand[i])

# Maximum iterations allowed in a search
ilimit = 30

# Maximum searches allowed per problem
slimit = 100

# Solution tolerance
tol = 1e-6

For the solver list, we aren't bothering with a wrapper class since we don't care about the stats, only the speed.

In [30]:
speed_solvers = [
    lambda Tep: ets.ik_NR(
        Tep,
        ilimit=ilimit,
        slimit=slimit,
        tol=tol,
        mask=we,
        pinv=True,
        joint_limits=True,
    ),
    lambda Tep: ets.ik_GN(
        Tep,
        ilimit=ilimit,
        slimit=slimit,
        tol=tol,
        mask=we,
        pinv=True,
        joint_limits=True,
    ),
    lambda Tep: ets.ik_LM(
        Tep,
        ilimit=ilimit,
        slimit=slimit,
        tol=tol,
        mask=we,
        k=1e-4,
        joint_limits=True,
        method="wampler",
    ),
    lambda Tep: ets.ik_LM(
        Tep,
        q0=None,
        ilimit=ilimit,
        slimit=slimit,
        tol=tol,
        mask=we,
        k=0.1,
        joint_limits=True,
        method="chan",
    ),
    lambda Tep: ets.ik_LM(
        Tep,
        ilimit=ilimit,
        slimit=slimit,
        tol=tol,
        mask=we,
        k=0.0001,
        joint_limits=True,
        method="sugihara",
    ),
]

speed_names = [
    "Newton Raphson (pinv=True)",
    "Gauss Newton (pinv=True)",
    "LM Wampler 1e-4",
    "LM Chan 0.1",
    "LM Sugihara 0.0001",
]

times = []

In [31]:
for name, solver in zip(speed_names, speed_solvers):
    print(f"Solving with {name}")

    start = time.time()

    for i in range(speed_problems):
        solver(Tep[i])

    total_time = time.time() - start
    times.append(total_time)


print(f"\nNumerical Inverse Kinematics Methods Times Compared over {speed_problems} problems\n")

table = ANSITable(
    "Method",
    "Total Time (s)",
    "Average Time per Solution (μs)",
    border="thin",
)

for name, t in zip(speed_names, times):
    table.row(
        name,
        np.round(t, 4),
        np.round((t / speed_problems) * 1e6, 4),
    )

table.print()

Solving with Newton Raphson (pinv=True)
Solving with Gauss Newton (pinv=True)
Solving with LM Wampler 1e-4
Solving with LM Chan 0.1
Solving with LM Sugihara 0.0001

Numerical Inverse Kinematics Methods Times Compared over 10000 problems

┌───────────────────────────┬────────────────┬────────────────────────────────┐
│                    Method │ Total Time (s) │ Average Time per Solution (μs) │
├───────────────────────────┼────────────────┼────────────────────────────────┤
│Newton Raphson (pinv=True) │         32.299 │                       3229.904 │
│  Gauss Newton (pinv=True) │        44.9978 │                      4499.7833 │
│           LM Wampler 1e-4 │        10.4467 │                      1044.6699 │
│               LM Chan 0.1 │         1.1382 │                        113.816 │
│        LM Sugihara 0.0001 │         1.5778 │                       157.7755 │
└───────────────────────────┴────────────────┴────────────────────────────────┘

